In [1]:
load_model = 'convnext_large_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


## load the convnext_large based model

In [4]:
convnext_large_loaded = models.convnext_large(pretrained=True)

for param in convnext_large_loaded.parameters():
    param.requires_grad = False

    
# densenet121_loaded.classifier = nn.Sequential(*list(model_densenet121.classifier.children())[:-1] + [nn.Linear(in_features=1024, out_features=120, bias=True)])
convnext_large_loaded.classifier[2] = nn.Linear(in_features=1536, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
convnext_large_loaded = convnext_large_loaded.to(device)



convnext_large_loaded.load_state_dict(torch.load('convnext_large_based_model2.pt'), strict=False)
convnext_large_loaded.eval()

ConvNeXt(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=192, out_features=768, bias=True)
          (4): GELU()
          (5): Linear(in_features=768, out_features=192, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=192, out

In [5]:
model = convnext_large_loaded

In [6]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

per_class_accuracy

{'Pomeranian': 0.0,
 'basset': 100.0,
 'Japanese spaniel': 100.0,
 'Bouvier des Flandres': 0.0,
 'German shepherd': 0.0,
 'Brabancon griffon': 0.0,
 'Blenheim spaniel': 100.0,
 'kelpie': 0.0,
 'Shih Tzu': 100.0,
 'curly coated retriever': 0.0,
 'Lakeland terrier': 0.0,
 'Irish setter': 0.0,
 'German short haired pointer': 0.0,
 'flat coated retriever': 0.0,
 'Irish water spaniel': 0.0,
 'Dandie Dinmont': 0.0,
 'clumber': 0.0,
 'Yorkshire terrier': 0.0,
 'schipperke': 0.0,
 'briard': 0.0,
 'miniature poodle': 0.0,
 'chow': 0.0,
 'Saluki': 0.0,
 'boxer': 0.0,
 'Pekinese': 100.0,
 'komondor': 0.0,
 'Brittany spaniel': 0.0,
 'black and tan coonhound': 100.0,
 'Appenzeller': 0.0,
 'Eskimo dog': 0.0,
 'giant schnauzer': 0.0,
 'Old English sheepdog': 0.0,
 'Chihuahua': 100.0,
 'miniature schnauzer': 0.0,
 'redbone': 0.0,
 'wire haired fox terrier': 0.0,
 'Irish wolfhound': 0.0,
 'dhole': 0.0,
 'Pembroke': 0.0,
 'keeshond': 0.0,
 'kuvasz': 0.0,
 'Sussex spaniel': 0.0,
 'Samoyed': 0.0,
 'malino

In [7]:
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

13.16702279202279

### Top 10 Breeds with least accuracy

In [8]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Pomeranian', 0.0),
 ('Bouvier des Flandres', 0.0),
 ('German shepherd', 0.0),
 ('Brabancon griffon', 0.0),
 ('kelpie', 0.0),
 ('curly coated retriever', 0.0),
 ('Lakeland terrier', 0.0),
 ('Irish setter', 0.0),
 ('German short haired pointer', 0.0),
 ('flat coated retriever', 0.0)]

### Top 15 Breeds with most accuracy

In [9]:
k = 15
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('basset', 100.0),
 ('Japanese spaniel', 100.0),
 ('Blenheim spaniel', 100.0),
 ('Shih Tzu', 100.0),
 ('Pekinese', 100.0),
 ('black and tan coonhound', 100.0),
 ('Chihuahua', 100.0),
 ('bloodhound', 100.0),
 ('Rhodesian ridgeback', 100.0),
 ('Walker hound', 100.0),
 ('Afghan hound', 100.0),
 ('papillon', 100.0),
 ('Maltese dog', 96.15384615384616),
 ('beagle', 95.0),
 ('bluetick', 94.44444444444444)]